# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

'0.0.60'

In [2]:
import os
DayFolder=os.getcwd()
meta_path=DayFolder +  '\MetaData'
spectra_path=DayFolder + '\Spectra'

## Load in specra, and combine

In [3]:
import os.path
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,Unnamed: 0,filename,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,K21-59-FI#1_CRR_DiadFit,103.058246,NaN,NaN,1286.035114,0.00436,592.273795,1286.035114,607.861450,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,K21-59-FI#2,103.010689,0.01727,0.012237,1286.157463,0.00919,227.669921,1286.157463,201.585656,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,K21-59-FI#3_CRR_DiadFit,103.046333,0.00848,0.006020,1286.074091,0.00462,570.408880,1286.074091,582.296696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,K21-61-FI#1-rep1_CRR_DiadFit,103.310206,0.04220,0.031913,1285.431901,0.02910,66.022125,1285.431901,82.081753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,K21-61-FI#1-rep2_CRR_DiadFit,103.304614,0.00626,0.004443,1285.435763,0.00286,607.809997,1285.435763,895.852963,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'FG', 'Cap','window'],
 sort=True)
diad_meta

['K21-59-FI#1-IMG (1).txt',
 'K21-59-FI#1-IMG.txt',
 'K21-59-FI#1.txt',
 'K21-59-FI#2-IMG.txt',
 'K21-59-FI#2.txt',
 'K21-59-FI#3-IMG.txt',
 'K21-59-FI#3.txt',
 'K21-59-IMG5x.txt',
 'K21-60-IMG5x.txt',
 'K21-61-FI#1-IMG.txt',
 'K21-61-FI#1-rep1.txt',
 'K21-61-FI#1-rep2.txt',
 'K21-61-FI#2-IMG.txt',
 'K21-61-FI#2.txt',
 'K21-61-FI#3-IMG.txt',
 'K21-61-FI#3.txt',
 'K21-61-IMG5x.txt',
 'K21-62-FI#1-IMG.txt',
 'K21-62-FI#1-nodiad.txt',
 'K21-62-FI#2-IMG.txt',
 'K21-62-FI#2-nodiad.txt',
 'K21-62-IMG5x.txt',
 'K21-63-FI#1-IMG.txt',
 'K21-63-FI#1.txt',
 'K21-63-IMG5x.txt',
 'K21-64-FI#1-IMG.txt',
 'K21-64-FI#1.txt',
 'K21-64-FI#2-IMG.txt',
 'K21-64-FI#2.txt',
 'K21-64-FI#3-IMG.txt',
 'K21-64-FI#3.txt',
 'K21-64-FI#4-IMG.txt',
 'K21-64-FI#4.txt',
 'K21-64-FI#5-IMG.txt',
 'K21-64-FI#5.txt',
 'K21-64-FI#6-IMG.txt',
 'K21-64-FI#6.txt',
 'K21-64-FI#7-IMG.txt',
 'K21-64-FI#7.txt',
 'K21-64-FI#8-IMG.txt',
 'K21-64-FI#8.txt',
 'K21-64-IMG5x.txt']

## Now get all the important information out of the metadata files

In [6]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=False)
meta['filename'].iloc[0]

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:01<00:00, 34.58it/s]


Done


'K21-59-FI#1.txt'

## Then get a simple file name you can stitch with the spectra

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'K21-59-FI#1'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [9]:
file_s

array(['K21-59-FI#1', 'K21-59-FI#2', 'K21-59-FI#3', 'K21-61-FI#1-rep1',
       'K21-61-FI#1-rep2', 'K21-63-FI#1', 'K21-64-FI#2', 'K21-64-FI#8',
       'K21-61-FI#2', 'K21-61-FI#3', 'K21-64-FI#1', 'K21-64-FI#3',
       'K21-64-FI#4', 'K21-64-FI#5', 'K21-64-FI#6', 'K21-64-FI#7'],
      dtype='<U16')

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
from pickle import load
import pickle
import os
pickle_path=os.getcwd()
file_pkl=pickle_path + '/' + 'Ne_Line_Regression_Example_cali.pck'
with open(file_pkl, 'rb') as file_handle:
    my_loaded_model = pickle.load(file_handle)

In [12]:
Ne_corr=pf.calculate_Ne_corr_std_err_values(pickle_str='polyfit_data.pkl', 
    new_x=df_combo['sec since midnight'], CI=0.67)
Ne_corr.head()


1


,time,preferred_values,lower_values,upper_values
0,43795.0,0.997845,0.997840,0.997851
1,44393.0,0.997837,0.997832,0.997843
2,45146.0,0.997828,0.997823,0.997834
3,45944.0,0.997819,0.997814,0.997824
4,46247.0,0.997816,0.997810,0.997821


In [13]:

split_err=pf.propagate_errors_for_splitting(Ne_corr, df_combo)
df_combo_out=df_combo.copy()
df_combo_out.insert(1, 'Corrected_Splitting', df_combo['Splitting']*Ne_corr['preferred_values'])
df_combo_out.insert(2, 'Corr_Split+1σ', df_combo_out['Corrected_Splitting']+split_err)
df_combo_out.insert(3, 'Corr_Split-1σ',df_combo_out['Corrected_Splitting']-split_err)
df_combo_out.insert(4, 'Corr_Split_1σ_val', split_err)
df_combo_out.to_excel('FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [14]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None
Sec_Phases.head()

,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb
0,K21-63-FI#1.txt,1150.651590,74.099762,53.038837,Spline,NaN,NaN,NaN
1,K21-64-FI#1.txt,1150.681087,623.983358,433.118641,Spline,NaN,NaN,NaN
2,K21-64-FI#2.txt,1150.681087,342.336440,232.573817,Spline,NaN,NaN,NaN
3,K21-64-FI#3.txt,1150.651590,1075.733101,718.895719,Spline,NaN,NaN,NaN
4,K21-64-FI#5.txt,1150.651590,764.715578,491.481742,Spline,NaN,NaN,NaN


In [15]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    file_sec_phase

good job, no duplicate file names


In [16]:
file_sec_phase

array(['K21-63-FI#1', 'K21-64-FI#1', 'K21-64-FI#2', 'K21-64-FI#3',
       'K21-64-FI#5', 'K21-64-FI#6', 'K21-64-FI#7', 'K21-64-FI#8',
       'K21-61-FI#1-rep2'], dtype=object)

In [17]:
df_combo['name_for_matching']

0          K21-59-FI#1
1          K21-59-FI#2
2          K21-59-FI#3
3     K21-61-FI#1-rep1
4     K21-61-FI#1-rep2
5          K21-63-FI#1
6          K21-64-FI#2
7          K21-64-FI#8
8          K21-61-FI#2
9          K21-61-FI#3
10         K21-64-FI#1
11         K21-64-FI#3
12         K21-64-FI#4
13         K21-64-FI#5
14         K21-64-FI#6
15         K21-64-FI#7
Name: name_for_matching, dtype: object

In [18]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo_out.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo_out

### Add some more useful columns

In [19]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))

In [20]:
df_combo_sec_phase.to_excel('FI_fitting_w_sec_phases.xlsx')

In [21]:
df_combo_sec_phase.head()

,Unnamed: 0,Corrected_Splitting,Corr_Split+1σ,Corr_Split-1σ,Corr_Split_1σ_val,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,...,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb,Carb_Diad_Ratio,SO2_Diad_Ratio
0,0,102.836162,102.840549,102.831774,0.004388,K21-59-FI#1_CRR_DiadFit,103.058246,NaN,NaN,1286.035114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,102.787916,102.800139,102.775692,0.012224,K21-59-FI#2,103.010689,0.01727,0.012237,1286.157463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,102.822535,102.828568,102.816502,0.006033,K21-59-FI#3_CRR_DiadFit,103.046333,0.00848,0.006020,1286.074091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,103.084886,103.116734,103.053038,0.031848,K21-61-FI#1-rep1_CRR_DiadFit,103.310206,0.04220,0.031913,1285.431901,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,103.078963,103.083430,103.074495,0.004467,K21-61-FI#1-rep2_CRR_DiadFit,103.304614,0.00626,0.004443,1285.435763,...,K21-61-FI#1-rep2.txt,NaN,NaN,NaN,NaN,1090.919723,900.825267,43.734455,0.387942,NaN
